## Wrangle it up!

This notebook will only focus on cleaning up the data, and creating a file that would be used for analysis.

In [1]:
#load libraries
import pandas as pd 
import numpy as np

In [2]:
#load all the csv files
jan = pd.read_csv("airlines/january_2019.csv")
feb = pd.read_csv("airlines/february_2019.csv")
mar = pd.read_csv("airlines/march_2019.csv")
apr = pd.read_csv("airlines/april_2019.csv")
may = pd.read_csv("airlines/may_2019.csv")
jun = pd.read_csv("airlines/june_2019.csv")
jul = pd.read_csv("airlines/july_2019.csv")
aug = pd.read_csv("airlines/august_2019.csv")
sep = pd.read_csv("airlines/september_2019.csv")
october = pd.read_csv("airlines/october_2019.csv")
nov = pd.read_csv("airlines/november_2019.csv")
dec = pd.read_csv("airlines/december_2019.csv")

#reference tables
operating_airline = pd.read_csv("operating_airline.csv") #airline code to name of airline

cancellation_code = pd.read_csv("cancellation_code.csv")
cancellation_code.iloc[-1] = ["None", "Did Not Cancel"]
cancellation_code.columns = ["CANCELLATION_CODE","CANCELLATION_DESCRIPTION"]

airport_id = pd.read_csv("airport_id.csv")
airport_id["AIRPORT_NAME"] = airport_id["Description"].str.split(":", expand = True)[1]

#concatenate into a single dataframe
df = pd.concat([jan, feb, mar, apr, may, jun, jul, aug, sep, october, nov, dec])

## Column Descriptions

1. FL_DATE = Flight date (yyyymmdd)
2. OP_UNIQUE_CARRIER = Operating Airline Code. Use operating_airline.csv to find the airline name 
3. OP_CARRIER_FL_NUM = Flight number operating airline
4. ORIGIN_AIRPORT_ID = Origin Airport, Airport ID
5. ORIGIN = Origin Airport
6. ORIGIN_CITY_NAME = Origin City Name
7. DEST_AIRPORT_ID = Destination Airport, Airport ID
8. DEST = Destination airport
9. DEST_CITY_NAME = Destination City Name
10. CRS_DEP_TIME = PLANNED departure time (local time: hhmm)
11. DEP_TIME = ACTUAL departure time (local time: hhmm)
12. DEP_DELAY = Difference in minutes between actual and planned departure time.
13. DEP_DELAY_NEW = Difference in minutes, but all early departed flights are set to 0
14. DEP_DEL15 = Departure delay indicator, 15 minutes or more
15. DEP_DEL_GROUP = Departure delay intervals, every 15 minutes
14. CRS_ARR_TIME = PLANNED arrival time (local time: hhmm)
15. ARR_TIME = ACTUAL arrival time (local time: hhmm)
16. ARR_DELAY = Difference in minutes between actual and planned arrival time.
17. ARR_DELAY_NEW = Difference in minutes, but all early arrived flights are set to 0
18. ARR_DEL15 = Arrival delay indicator, 15 minutes or more
19. CANCELLED = Cancelled flight indicator
20. CANCELLATION_CODE = Specifies the reason for cancelation
21. DIVERTED = Diverted flight indicator
22. DUP = Duplicate, if the flight has been swapped
23. CRS_ELAPSED_TIME = CRS elapsed time of flight
23. ACTUAL_ELAPSED_TIME = Elapsed time of flight in minutes
24. AIR_TIME = flight time in minutes
25. FLIGHTS = Number of flights
26. DISTANCE = Distance between airports
27. DISTANCE GROUP = Distance intervals, for Flight Segment
28. CARRIER_DELAY = Carrier delay, in minutes
29. WEATHER_DELAY = Weather delay, in minutes
30. NAS_DELAY = Natoinal air system delay, in minutes
31. SECURITY_DELAY = Security delay, in minutes
32. LATE_AIRCRAFT_DELAY = Late aircraft delay, in minutes
33. DIV_AIRPORT_LANDINGS = Number of diverted landings

# Data Cleaning

The following will be done to clean up the data:
1. Check for missing data
2. Understand why some of the data is missing
3. Fill in missing data
4. Adding in features.

In [3]:
#check for missing values
df.isnull().sum()*100/len(df)

FL_DATE                  0.000000
OP_UNIQUE_CARRIER        0.000000
OP_CARRIER_FL_NUM        0.000000
ORIGIN_AIRPORT_ID        0.000000
ORIGIN                   0.000000
ORIGIN_CITY_NAME         0.000000
DEST_AIRPORT_ID          0.000000
DEST                     0.000000
DEST_CITY_NAME           0.000000
CRS_DEP_TIME             0.000000
DEP_TIME                 1.827728
DEP_DELAY                1.828025
DEP_DELAY_NEW            1.828025
DEP_DEL15                1.828025
DEP_DELAY_GROUP          1.828025
CRS_ARR_TIME             0.000000
ARR_TIME                 1.939226
ARR_DELAY                2.155546
ARR_DELAY_NEW            2.155546
ARR_DEL15                2.155546
ARR_DELAY_GROUP          2.155546
CANCELLED                0.000000
CANCELLATION_CODE       98.101396
DIVERTED                 0.000000
DUP                      0.000000
CRS_ELAPSED_TIME         0.000124
ACTUAL_ELAPSED_TIME      2.155546
AIR_TIME                 2.155546
FLIGHTS                  0.000000
DISTANCE      

Observations:

1. A large majority of the missing data is coming from features that are related to cancellation of flights(CANCELLATION_CODE, CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURTY_DELAY, AIRCRAFT_DELAY). The reason for the missing data is most likely because the flights didn't get delayed.
2. Unnamed: 36 doesn't hold any information and can be deleted.
3. Departure and Arrival features also have missing data, but are all less than 5%. 

Next thing to do would be to first delete "Unnamed: 36" column and then try to check on the missing data and find any patterns. Once patterns have been set, fill in the missing values.

## Delete Unnecessary Columns

In [4]:
#delete as it does not have anything inside
del df["Unnamed: 36"]

## Understand Missing Data

In [5]:
df[df["DEP_TIME"].isnull()][["CRS_DEP_TIME", "DEP_TIME", "DEP_DELAY", "DEP_DELAY_NEW", "DEP_DEL15"]].sample(5)

,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15
572355,2121,NaN,NaN,NaN,NaN
50871,910,NaN,NaN,NaN,NaN
533951,745,NaN,NaN,NaN,NaN
68552,905,NaN,NaN,NaN,NaN
282700,1115,NaN,NaN,NaN,NaN


The 5 random samples where DEP_TIME is missing, it shows that "DEP_DELAY", "DEP_DELAY_NEW" and "DEP_DEL15" are cannot be found as well. In thise case, "DEP_TIME" will asume to have departed on time (DEP_TIME = CRS_DEP_TIME). For the rest of the missing columns, they will all be filled with 0, as it assumes that all flights with missing data have departed on time.

### Departed and Arrived Flights

In [6]:
#fill in departure performance columns
df["DEP_TIME"] = df["DEP_TIME"].fillna(df["CRS_DEP_TIME"]).astype("int") 
df["DEP_DELAY"] = df["DEP_DELAY"].fillna(0)
df["DEP_DELAY_NEW"] = df["DEP_DELAY_NEW"].fillna(0)
df["DEP_DEL15"] = df["ARR_DEL15"].fillna(0)

#fill in departure performance columns
df["ARR_TIME"] = df["ARR_TIME"].fillna(df["CRS_ARR_TIME"]).astype("int")
df["ARR_DELAY"] = df["ARR_DELAY"].fillna(0)
df["ARR_DELAY_NEW"] = df["ARR_DELAY_NEW"].fillna(0)
df["ARR_DEL15"] = df["ARR_DEL15"].fillna(0)

### Cancelled Flights

In [7]:
df[df["CANCELLATION_CODE"].isnull() == True]["CANCELLED"].value_counts()

0.0    7938055
Name: CANCELLED, dtype: int64

The 0 value means that the flight hasn't been cancelled. This shows that the cancelled flights would only reflect when the value of the cancelled column is equal to 1.

In [8]:
#fill in cancellation code with None
df["CANCELLATION_CODE"] = df["CANCELLATION_CODE"].fillna("None")

Filled in missing values of cancellation code to "None" to signify that the flight hasn't been cancelled.

In [9]:
df[df["CARRIER_DELAY"].isnull() == True][["DEP_DEL15", "ARR_DEL15", "CARRIER_DELAY", "WEATHER_DELAY","NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY"]].sample(10)

,DEP_DEL15,ARR_DEL15,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
572069,0.0,0.0,NaN,NaN,NaN,NaN,NaN
46603,0.0,0.0,NaN,NaN,NaN,NaN,NaN
256415,0.0,0.0,NaN,NaN,NaN,NaN,NaN
208242,0.0,0.0,NaN,NaN,NaN,NaN,NaN
619362,0.0,0.0,NaN,NaN,NaN,NaN,NaN
520248,0.0,0.0,NaN,NaN,NaN,NaN,NaN
236521,0.0,0.0,NaN,NaN,NaN,NaN,NaN
443078,0.0,0.0,NaN,NaN,NaN,NaN,NaN
551900,0.0,0.0,NaN,NaN,NaN,NaN,NaN
592458,0.0,0.0,NaN,NaN,NaN,NaN,NaN


Sample set of missing "CARRIER_DELAY" data, shows that both departure and arrival columns didn't get delayed. Then assume that all delay features will be at 0.

In [10]:
#fill in missing values from delay parameters to 0 as they assume to have 0 to very little delay( <15 minutes)
df[["CARRIER_DELAY", "WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY"]] = df[["CARRIER_DELAY", "WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY"]].fillna(0)

## Turn Date columns into Datetime

Now all the columns have been cleaned up, turn the necessary columns into datetime.

In [11]:
df[["CRS_DEP_TIME", "CRS_ARR_TIME"]].sample(5)

,CRS_DEP_TIME,CRS_ARR_TIME
75542,1150,1750
198663,1817,2010
185493,857,1028
672553,1155,1515
292639,1228,1326


CRS_DEP_TIME and CRS_ARR_TIME are in an integer format, and uses military time.

In [12]:
df[(df["CRS_DEP_TIME"].astype("str").str.len() < 3) | (df["CRS_ARR_TIME"].astype("str").str.len() < 3) ][["CRS_DEP_TIME", "CRS_ARR_TIME"]].sample(10)

,CRS_DEP_TIME,CRS_ARR_TIME
72718,2140,10
77151,2110,8
603123,45,515
214428,2231,44
425377,2314,7
655753,2140,5
186559,2233,15
63485,2146,38
501278,2215,39
406241,2000,30


Apparently, there are instances in the dataset where there are only 2 digits or less, this will be assumed that these times are just after midnight. 

In [13]:
df[(df["CRS_DEP_TIME"].astype("str").str.len() > 4) | (df["CRS_ARR_TIME"].astype("str").str.len() > 4)].shape

(0, 36)

Any digits that are more than 5 does not exist, then we assume that there is no error under these columns.

In [14]:
#create datetime columns and convert string into datetime dtype
def int_to_time(time):
    if len(time) == 1: #if there's only a single digit
        normal_len = str(0) + str(0) + str(0) + time
        return normal_len[:2] + ":" + normal_len[2:] + ":" + "00"
    
    elif len(time) == 2:
        normal_len = str(0) + str(0) + time
        return normal_len[:2] + ":" + normal_len[2:] + ":" + "00"
    
    elif len(time) == 3:
        normal_len =  str(0) + time
        return normal_len[:2] + ":" + normal_len[2:] + ":" + "00"
    
    else: return time[:2] + ":" + time[2:] + ":" + "00" #if there are 4 digits

In order to turn the time column, which is an integer dtype, into a date dtype is to transform the format from xxxx to hh:mm:ss. Since the length of the time column comes into two forms: which is 3 and 4 (example: 340 and 1229) then these will be pre-processed differently.

1. Count length of digits
    1. When length = 1: add in 000 at the start of the string
    2. When length = 2: add in 00 at the start of the string
    3. When length = 3: add in 0 at the start of the string
    4. When length = 4: add nothing in the string
2. Then add in a ":" mark after every 2 characters
3. After the second ":" mark, add in "00" to signify seconds

In [15]:
#departure 
#crs
df["CRS_DEP_TIME"] = df["CRS_DEP_TIME"].astype("str").apply(int_to_time)

df["CRS_DEP_DATE"] = pd.to_datetime(df["FL_DATE"]) + pd.to_timedelta(df["CRS_DEP_TIME"])
df["CRS_DEP_MONTH"] = df["CRS_DEP_DATE"].dt.month
df["CRS_DEP_DAY"] = df["CRS_DEP_DATE"].dt.day
df["CRS_DEP_DOW"] = df["CRS_DEP_DATE"].dt.dayofweek
df["CRS_DEP_HOUR"] = df["CRS_DEP_DATE"].dt.hour

#actual
df["DEP_TIME"] = df["DEP_TIME"].astype("str").apply(int_to_time)

df["DEP_DATE"] = pd.to_datetime(df["FL_DATE"]) + pd.to_timedelta(df["DEP_TIME"])
df["DEP_MONTH"] = df["DEP_DATE"].dt.month
df["DEP_DAY"] = df["DEP_DATE"].dt.day
df["DEP_DOW"] = df["DEP_DATE"].dt.dayofweek
df["DEP_HOUR"] = df["DEP_DATE"].dt.hour


#arrival
#crs
df["CRS_ARR_TIME"] = df["CRS_ARR_TIME"].astype("str").apply(int_to_time)

df["CRS_ARR_DATE"] = pd.to_datetime(df["FL_DATE"]) + pd.to_timedelta(df["CRS_ARR_TIME"])
df["CRS_ARR_MONTH"] = df["CRS_ARR_DATE"].dt.month
df["CRS_ARR_DAY"] = df["CRS_ARR_DATE"].dt.day
df["CRS_ARR_DOW"] = df["CRS_ARR_DATE"].dt.dayofweek
df["CRS_ARR_HOUR"] = df["CRS_ARR_DATE"].dt.hour

#actual
df["ARR_TIME"] = df["ARR_TIME"].astype("str").apply(int_to_time)

df["ARR_DATE"] = pd.to_datetime(df["FL_DATE"]) + pd.to_timedelta(df["ARR_TIME"])
df["ARR_MONTH"] = df["ARR_DATE"].dt.month
df["ARR_DAY"] = df["ARR_DATE"].dt.day
df["ARR_DOW"] = df["ARR_DATE"].dt.dayofweek
df["ARR_HOUR"] = df["ARR_DATE"].dt.hour

Cleaned up both the CRS and actual time columns under the departure and arrival features. 

## Merge the datasets

In [16]:
#merge datasets
df = df.merge(operating_airline, left_on = "OP_UNIQUE_CARRIER", right_on = "Code")
df = df.merge(cancellation_code, how = "left", on = "CANCELLATION_CODE")
df = df.merge(airport_id[["Code", "AIRPORT_NAME"]], left_on = "ORIGIN_AIRPORT_ID", right_on = "Code" )
df = df.merge(airport_id[["Code", "AIRPORT_NAME"]], left_on = "DEST_AIRPORT_ID", right_on = "Code" )

df = df.rename(columns = {"AIRPORT_NAME_x":"ORIGIN_AIRPORT", "AIRPORT_NAME_y":"DEST_AIRPORT"})
df = df.drop(["Code_x", "Code_y"], axis = 1)

#filter out flight data with main passenger
passenger_airlines = "Alaska Airlines|Allegiant|American Airlines|Delta|Frontier Airlines|Hawaiian|JetBlue|Southwest|Spirit Air|United Air Lines"

major_airlines = df[df["Description"].str.match(passenger_airlines) == True]

#top 40 busiest airlines - https://en.wikipedia.org/wiki/List_of_the_busiest_airports_in_the_United_States - used IATA code
airport_busiest = ["ATL", "DFW", "DEN", "ORD", "LAX", "CLT", "LAS", "PHX", "MCO", "SEA", "MIA", "IAH", "FLL", "SFO",
                  "JFK", "MSP", "EWR", "DTW", "BOS", "SLC", "PHL", "BWI", "TPA", "SAN", "MDW", "IAD", "BNA", "LGA",
                  "DAL", "DCA", "PDX", "HNL", "HOU", "AUS", "STL", "RSW", "SMF", "MSY", "RDU", "SJU"]

major_airlines = major_airlines[major_airlines["ORIGIN"].isin(airport_busiest)]
major_airlines = major_airlines[major_airlines["DEST"].isin(airport_busiest)]

In [17]:
major_airlines.to_csv("final.csv", index = False)

# References:


1. [Data Source](https://www.transtats.bts.gov/DL_SelectFields.asp?gnoyr_VQ=FGK&QO_fu146_anzr=b0-gvzr)
2. [Inspiration](https://fivethirtyeight.com/features/how-we-found-the-fastest-flights/)
3. [Why is my flight delayed?](https://www.claimcompass.eu/blog/why-is-my-flight-delayed/#:~:text=According%20to%20the%20Bureau%20of,by%2015%20minutes%20or%20more.&text=Let's%20start%20with%20the%2015%20most%20frequent%20reasons%20for%20those%20delays)
4. [Major Airlines](https://en.wikipedia.org/wiki/Major_airlines_of_the_United_States)